# Solar PV Short-Term Forecasting - Data Preprocessing

This notebook loads, merges, resamples, cleans, and prepares solar plant generation and weather sensor data.

## 1. Import Libraries

In [1]:
import pandas as pd

## 2. Load Data

In [2]:
# Load Plant 1 Generation Data with datetime parsing
generation_df = pd.read_csv('data/Plant_1_Generation_Data.csv', parse_dates=['DATE_TIME'])

# Load Plant 1 Weather Sensor Data with datetime parsing
weather_df = pd.read_csv('data/Plant_2_Weather_Sensor_Data.csv', parse_dates=['DATE_TIME'])

C:\Users\DELL\AppData\Local\Temp\ipykernel_168\2764739940.py:2: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  generation_df = pd.read_csv('data/Plant_1_Generation_Data.csv', parse_dates=['DATE_TIME'])


### Plant 1 Generation Data

In [3]:
# Display column names and data types
print("Column Names and Data Types:")
print(generation_df.dtypes)

Column Names and Data Types:
DATE_TIME      datetime64[ns]
PLANT_ID                int64
SOURCE_KEY             object
DC_POWER              float64
AC_POWER              float64
DAILY_YIELD           float64
TOTAL_YIELD           float64
dtype: object


In [4]:
# Display dataset shape
print("Dataset Shape:", generation_df.shape)

Dataset Shape: (68778, 7)


In [5]:
# Display first 5 rows
generation_df.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,2020-05-15,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0
1,2020-05-15,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0
2,2020-05-15,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0
3,2020-05-15,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0
4,2020-05-15,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0


### Plant 1 Weather Sensor Data

In [6]:
# Display column names and data types
print("Column Names and Data Types:")
print(weather_df.dtypes)

Column Names and Data Types:
DATE_TIME              datetime64[ns]
PLANT_ID                        int64
SOURCE_KEY                     object
AMBIENT_TEMPERATURE           float64
MODULE_TEMPERATURE            float64
IRRADIATION                   float64
dtype: object


In [7]:
# Display dataset shape
print("Dataset Shape:", weather_df.shape)

Dataset Shape: (3259, 6)


In [8]:
# Display first 5 rows
weather_df.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4136001,iq8k7ZNt4Mwm3w0,27.004764,25.060789,0.0
1,2020-05-15 00:15:00,4136001,iq8k7ZNt4Mwm3w0,26.880811,24.421869,0.0
2,2020-05-15 00:30:00,4136001,iq8k7ZNt4Mwm3w0,26.682055,24.427290,0.0
3,2020-05-15 00:45:00,4136001,iq8k7ZNt4Mwm3w0,26.500589,24.420678,0.0
4,2020-05-15 01:00:00,4136001,iq8k7ZNt4Mwm3w0,26.596148,25.088210,0.0


## 3. Merge Generation and Weather Data

In [9]:
# Print shapes before merging
print("Shape before merging:")
print(f"Generation Data: {generation_df.shape}")
print(f"Weather Data: {weather_df.shape}")

# Merge dataframes on DATE_TIME using inner join
merged_df = pd.merge(generation_df, weather_df, on='DATE_TIME', how='inner')

# Sort by DATE_TIME
merged_df = merged_df.sort_values('DATE_TIME').reset_index(drop=True)

# Check for duplicate timestamps
print(f"\nDuplicate timestamps before removal: {merged_df.duplicated(subset='DATE_TIME').sum()}")

# Remove duplicate timestamps if any
merged_df = merged_df.drop_duplicates(subset='DATE_TIME', keep='first')

# Print shape after merging
print(f"\nShape after merging: {merged_df.shape}")
print(f"Duplicate timestamps after removal: {merged_df.duplicated(subset='DATE_TIME').sum()}")

Shape before merging:
Generation Data: (68778, 7)
Weather Data: (3259, 6)

Duplicate timestamps before removal: 65554

Shape after merging: (3154, 12)
Duplicate timestamps after removal: 0


In [10]:
# Display first few rows of merged data
merged_df.head()

,DATE_TIME,PLANT_ID_x,SOURCE_KEY_x,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,PLANT_ID_y,SOURCE_KEY_y,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,4136001,iq8k7ZNt4Mwm3w0,27.004764,25.060789,0.0
21,2020-05-15 00:15:00,4135001,iCRJl6heRkivqQ3,0.0,0.0,0.0,7177992.0,4136001,iq8k7ZNt4Mwm3w0,26.880811,24.421869,0.0
42,2020-05-15 00:30:00,4135001,iCRJl6heRkivqQ3,0.0,0.0,0.0,7177992.0,4136001,iq8k7ZNt4Mwm3w0,26.682055,24.427290,0.0
63,2020-05-15 00:45:00,4135001,pkci93gMrogZuBj,0.0,0.0,0.0,7169102.0,4136001,iq8k7ZNt4Mwm3w0,26.500589,24.420678,0.0
84,2020-05-15 01:00:00,4135001,iCRJl6heRkivqQ3,0.0,0.0,0.0,7177992.0,4136001,iq8k7ZNt4Mwm3w0,26.596148,25.088210,0.0


## 4. Resample to Hourly Resolution

In [11]:
# Set DATE_TIME as index
merged_df['DATE_TIME'] = pd.to_datetime(merged_df['DATE_TIME'])
merged_df = merged_df.set_index('DATE_TIME')

# Resample to hourly frequency using mean
resampled_df = merged_df.resample('H').mean()

# Store shape before filtering
rows_before = resampled_df.shape[0]

# Remove rows where IRRADIATION == 0
resampled_df = resampled_df[resampled_df['IRRADIATION'] != 0]

# Calculate and report rows removed
rows_after = resampled_df.shape[0]
rows_removed = rows_before - rows_after
print(f"Rows removed (IRRADIATION == 0): {rows_removed}")
print(f"Rows remaining: {rows_after}")

C:\Users\DELL\AppData\Local\Temp\ipykernel_168\70548852.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  resampled_df = merged_df.resample('H').mean()


TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
# Display first 5 rows
resampled_df.head()

## 5. Handle Missing Values

In [ ]:
# Print percentage of missing values per column
print("Missing Values Percentage:")
missing_pct = (resampled_df.isnull().sum() / len(resampled_df)) * 100
print(missing_pct)

# Print shape before cleaning
print(f"\nShape before cleaning: {resampled_df.shape}")

In [ ]:
# Identify weather columns (all columns except DC_POWER and AC_POWER)
weather_cols = [col for col in resampled_df.columns if col not in ['DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD']]

# Apply linear interpolation to weather variables
resampled_df[weather_cols] = resampled_df[weather_cols].interpolate(method='linear')

# Drop rows with missing DC_POWER values
resampled_df = resampled_df.dropna(subset=['DC_POWER'])

# Print shape after cleaning
print(f"Shape after cleaning: {resampled_df.shape}")

In [ ]:
# Verify missing values after cleaning
print(f"Missing values after cleaning:")
print(resampled_df.isnull().sum())

In [ ]:
# Display first 5 rows
resampled_df.head()

## 6. Save Cleaned Dataset

In [ ]:
# Save to CSV with datetime index preserved
resampled_df.to_csv('data/solar_pv_clean_hourly.csv')

# Print final dataset information
print(f"\nFinal dataset shape: {resampled_df.shape}")
print(f"Date range: {resampled_df.index.min()} to {resampled_df.index.max()}")
print(f"\nDataset saved to: data/solar_pv_clean_hourly.csv")

## 7. Sanity Check

In [ ]:
# Print column names
print("Column Names:")
print(resampled_df.columns.tolist())

In [ ]:
# Print date range
print(f"Date Range:")
print(f"Start: {resampled_df.index.min()}")
print(f"End: {resampled_df.index.max()}")

In [ ]:
# Print basic descriptive statistics for DC_POWER
print(f"DC_POWER Descriptive Statistics:")
print(resampled_df['DC_POWER'].describe())